In [4]:
import simpy
import random

class Job:
    def __init__(self, name, arrtime, duration):
        self.name = name
        self.arrtime = arrtime
        self.duration = duration

    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)

def SJF( job ):
    return job.duration

''' A server
 - env: SimPy environment
 - strat: - FIFO: First In First Out
          - SJF : Shortest Job First
'''
class Server:
    def __init__(self, env, strat = 'FIFO'):
        self.strat = strat
        self.Jobs = list(())
        self.serversleeping = None
        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        ''' register a new server process '''
        env.process( self.serve(env) )

    def serve(self, env):
        while True:
            ''' do nothing, just change server to idle
              and then yield a wait event which takes infinite time
            '''
            if len( self.Jobs ) == 0 :
                self.serversleeping = env.process( self.waiting( env ))
                t1 = env.now
                yield self.serversleeping
                ''' accumulate the server idle time'''
                self.idleTime += env.now - t1
            else:
                ''' get the first job to be served'''
                if self.strat == 'SJF':
                    self.Jobs.sort( key = SJF )
                    j = self.Jobs.pop( 0 )
                else: # FIFO by default
                    j = self.Jobs.pop( 0 )

                ''' sum up the waiting time'''
                self.waitingTime += env.now - j.arrtime
                ''' yield an event for the job finish'''
                yield env.timeout( j.duration )

    def waiting(self, env):
        try:
            print( 'Server is idle at %d' % env.now )
            yield env.timeout(1000)
        except simpy.Interrupt as i:
            print('A new job comes. Server waken up and works now at %d' % env.now )

class JobGenerator:
    def __init__(self, env, server, nrjobs = 2000, lam = .1, mu = .1):
        self.server = server
        self.nrjobs = nrjobs
        self.lam = lam
        self.mu = mu
        env.process( self.generatejobs(env) )

    def generatejobs(self, env):
        i = 1
        while True:
            '''yield an event for new job arrival'''
            job_interarrival = random.expovariate( self.lam )
            yield env.timeout( job_interarrival )

            ''' generate service time and add job to the list'''
            job_duration = random.expovariate( self.mu )
            self.server.Jobs.append( Job('Job %s' %i, env.now, job_duration) )
            print( 'job %d: t = %d, l = %d' %( i, env.now, job_duration ) )
            i += 1

            ''' if server is idle, wake it up'''
            if not self.server.serversleeping.triggered:
                self.server.serversleeping.interrupt( 'Wake up, please.' )

env = simpy.Environment()
MyServer = Server( env, 'FIFO' )
MyJobGenerator = JobGenerator( env, MyServer, 50, .5, .1 )
env.run( until = 100 )
print( 'Total waiting time     : %.2f' % MyServer.waitingTime )
print( 'Mean waiting time      : %.2f' % (MyServer.waitingTime/MyJobGenerator.nrjobs) )
print( 'Total server idle time : %.2f' % MyServer.idleTime )


Server is idle at 0
job 1: t = 6, l = 17
A new job comes. Server waken up and works now at 6
job 2: t = 7, l = 0
job 3: t = 11, l = 14
job 4: t = 15, l = 32
job 5: t = 16, l = 2
job 6: t = 21, l = 2
job 7: t = 23, l = 4
job 8: t = 24, l = 9
job 9: t = 28, l = 3
job 10: t = 30, l = 1
job 11: t = 39, l = 13
job 12: t = 40, l = 25
job 13: t = 44, l = 6
job 14: t = 44, l = 10
job 15: t = 49, l = 4
job 16: t = 49, l = 9
job 17: t = 50, l = 3
job 18: t = 52, l = 0
job 19: t = 52, l = 14
job 20: t = 53, l = 20
job 21: t = 54, l = 16
job 22: t = 56, l = 9
job 23: t = 61, l = 5
job 24: t = 69, l = 8
job 25: t = 70, l = 5
job 26: t = 71, l = 2
job 27: t = 72, l = 16
job 28: t = 73, l = 7
job 29: t = 74, l = 4
job 30: t = 76, l = 5
job 31: t = 77, l = 4
job 32: t = 85, l = 19
job 33: t = 85, l = 4
job 34: t = 87, l = 3
job 35: t = 87, l = 7
job 36: t = 88, l = 12
job 37: t = 96, l = 9
Total waiting time     : 440.94
Mean waiting time      : 8.82
Total server idle time : 6.05
